# ABS Monthly Arrivals Departures 3401

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd
import readabs as ra
from readabs import metacol as mc
import mgplot as mg

In [2]:
# local imports
from abs_helper import get_abs_data

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False

## Get data from ABS

In [3]:
abs_dict, meta, source, RECENT_ = get_abs_data("3401.0")
RECENT = pd.Period(RECENT_, freq="M")

plot_times = None, RECENT
plot_tags = "", "-recent"

Table 3401013 has no 'Index' sheet.
Table 3401014 has no 'Index' sheet.
Table 3401015 has no 'Index' sheet.
Table 3401016 has no 'Index' sheet.
Table 3401017 has no 'Index' sheet.


In [4]:
# data up-to-date to (year-month) ...
print(abs_dict["340101"].index[-1])

2025-03


## Total Overseas Arrivals and Departures

In [5]:
print(RECENT)

2020-12


In [6]:
def headline() -> None:
    """Headline charts"""

    plotable = (
        # [did, table]
        ("Number of movements ;  Total Arrivals ;", "340101"),
        ("Number of movements ;  Total Departures ;", "340102"),
    )

    plt_df = pd.DataFrame()
    for did, table in plotable:
        search = {
            did: mc.did,
            table: mc.table,
            # "Original": mc.stype,
        }
        _table, series_id, units = ra.find_abs_id(meta, search)
        plt_df[did.split(";")[-2]] = abs_dict[table][series_id]

    plt_df, units = ra.recalibrate(plt_df, units)

    mg.multi_start(
        plt_df,
        function=mg.line_plot_finalise,
        starts=[0, RECENT],
        title="Total Overseas Arrivals and Departures",
        ylabel=units,
        rfooter=source,
        lfooter="Australia. ",
        verbose=False,
        legend=True,
        show=SHOW,
    )


headline()

## Individual overseas movements

In [7]:
def plot_movements() -> None:
    """Headline charts"""

    tables = "340101", "340102"
    arrivals = meta.loc[meta[mc.table] == tables[0], mc.did]
    departures = meta.loc[meta[mc.table] == tables[1], mc.did]
    movements = arrivals, departures

    for movement, table in zip(movements, tables):
        data = abs_dict[table]
        for did in movement:
            search = {
                did: mc.did,
                table: mc.table,
                "Original": mc.stype,
            }
            _table, series_id, units = ra.find_abs_id(
                meta, search, exact_match=True, verbose=False
            )
            series, units = ra.recalibrate(data[series_id], units)

            mg.multi_start(
                series,
                starts=plot_times,
                function=mg.line_plot_finalise,
                title=did.split(";")[-2],
                ylabel=units,
                rfooter=source,
                lfooter="Australia. ",
                show=SHOW,
            )

            mg.postcovid_plot_finalise(
                series,
                title=did.split(";")[-2],
                ylabel=units,
                rfooter=source,
                lfooter="Australia. ",
                show=SHOW,
            )


plot_movements()

## Migration proxy

In [8]:
def migration_proxy() -> None:
    """Migration proxy charts"""

    plotable = (
        # [did, table]
        ("Number of movements ;  Permanent and Long-term Arrivals ;", "340101"),
        ("Number of movements ;  Permanent and Long-term Departures ;", "340102"),
    )

    plt_df = pd.DataFrame()
    for did, table in plotable:
        search = {
            did: mc.did,
            table: mc.table,
            "Original": mc.stype,
        }
        _table, series_id, units = ra.find_abs_id(meta, search)
        plt_df[did.split(";")[-2]] = abs_dict[table][series_id]

    YEAR = 12  # months
    plt_df = plt_df.rolling(window=YEAR, min_periods=YEAR).sum()
    net = plt_df[plt_df.columns[0]] - plt_df[plt_df.columns[1]]
    net, units = ra.recalibrate(net, units)

    mg.multi_start(
        net,
        function=mg.line_plot_finalise,
        starts=(None, RECENT),
        title="Net permanent and long-term overseas movements\n"
        f"(arrivals - departures), {YEAR}-month rolling sum",
        ylabel=units,
        rfooter=source,
        lfooter="Australia. ",
        y0=True,
        show=SHOW,
    )


migration_proxy()

## Finished

In [9]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-05-29 18:13:13

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

conda environment: n/a

Compiler    : Clang 20.1.0 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

matplotlib: 3.10.3
mgplot    : 0.1.1
readabs   : 0.0.29
pandas    : 2.2.3

Watermark: 2.5.0



In [10]:
print("Finished")

Finished
